In [ ]:
!pip install datasets transformers accelerate bitsandbytes peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 41.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("open-r1/codeforces-cots", "solutions")



README.md: 0.00B [00:00, ?B/s]

solutions/train-00000-of-00010.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

solutions/train-00001-of-00010.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

solutions/train-00002-of-00010.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

solutions/train-00003-of-00010.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

...

solutions/train-00006-of-00010.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

solutions/train-00007-of-00010.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

solutions/train-00008-of-00010.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

solutions/train-00009-of-00010.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47780 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'examples', 'note', 'editorial', 'prompt', 'generation', 'finish_reason', 'api_metadata', 'interaction_format', 'messages'],
        num_rows: 47780
    })
})

In [ ]:
ds['train'][0]['messages']

[{'content': 'You will be given a competitive programming problem. Please reason step by step about the solution, then provide a complete implementation in C++17.\n\nYour solution must read input from standard input (cin), write output to standard output (cout).\nDo not include any debug prints or additional output.\n\nPut your final solution within a single code block:\n```cpp\n<your code here>\n```\n\n# Problem\n\nYou are given an array $$$a$$$ of $$$n$$$ integers, where $$$n$$$ is odd.\n\nIn one operation, you will remove two adjacent elements from the array $$$a$$$, and then concatenate the remaining parts of the array. For example, given the array $$$[4,7,4,2,9]$$$, we can obtain the arrays $$$[4,2,9]$$$ and $$$[4,7,9]$$$ by the operations $$$[\\underline{4,7}, 4,2,9] \\to [4,2,9]$$$ and $$$[4,7,\\underline{4,2},9] \\to [4,7,9]$$$ respectively. However, we cannot obtain the array $$$[7,2,9]$$$ as it requires deleting non-adjacent elements $$$[\\underline{4},7,\\underline{4},2,9]$$

In [ ]:
!pip uninstall -y bitsandbytes # Force uninstall bitsandbytes
!pip install -U bitsandbytes # Reinstall the latest bitsandbytes

Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)


In [ ]:
import accelerate
!pip install -U bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # Import BitsAndBytesConfig
import torch # Import torch for dtype

model_name = "google/gemma-2-2b-it"   # or gemma-7b-it

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Create a BitsAndBytesConfig object
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Or "fp4"
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config, # Pass the quantization_config object
    device_map="auto",
    trust_remote_code=True
)


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length = 1536
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                       # very important for T4
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
gradient_checkpointing = True
model.gradient_checkpointing_enable()
# Enable gradient calculation for model inputs, crucial for gradient checkpointing with PEFT
model.enable_input_require_grads()
model.print_trainable_parameters()


trainable params: 3,194,880 || all params: 2,617,536,768 || trainable%: 0.1221


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="gemma-codeforces-cot-t4",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-4,
    warmup_ratio=0.03,
    num_train_epochs=1,
    logging_steps=20,
    save_steps=5000,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    remove_unused_columns=False,
    gradient_checkpointing=True,  # important for T4
)


In [ ]:
# define train_dataset and data_collator
from datasets import Dataset
from transformers import DataCollatorWithPadding

def preprocess_function(batch):
    tokenized_outputs = {
        'input_ids': [],
        'labels': [],
        'attention_mask': []
    }

    # Iterate over each example (conversation history) in the batch
    # Fix: Iterate by index to handle batch['messages'] being a Dataset object
    for i in range(len(batch['messages'])):
        messages_list = batch['messages'][i]

        # 1. Tokenize the full conversation (user + assistant turns)
        full_tokenized_output = tokenizer.apply_chat_template(
            messages_list,
            truncation=True,
            max_length=1536,
            padding=False,
            return_tensors=None, # Return as list of integers for DataCollator
            tokenize=True,
            add_generation_prompt=False, # We want the entire conversation for training input
            return_dict=True
        )

        full_input_ids = full_tokenized_output['input_ids']
        full_attention_mask = full_tokenized_output['attention_mask']

        # 2. Determine the length of the "user" portion for masking labels.
        user_prefix_messages = []
        for msg in messages_list:
            if msg['role'] == 'assistant':
                break # Stop collecting messages once the assistant's turn starts
            user_prefix_messages.append(msg)

        user_part_for_masking_output = tokenizer.apply_chat_template(
            user_prefix_messages,
            truncation=True,
            max_length=1536,
            padding=False,
            return_tensors=None,
            tokenize=True,
            add_generation_prompt=True, # Include the prompt for assistant's generation
            return_dict=True
        )

        user_prompt_token_length = len(user_part_for_masking_output['input_ids'])

        labels = full_input_ids.copy()

        for j in range(min(user_prompt_token_length, len(labels))):
            labels[j] = -100

        tokenized_outputs['input_ids'].append(full_input_ids)
        tokenized_outputs['labels'].append(labels)
        tokenized_outputs['attention_mask'].append(full_attention_mask)

    return tokenized_outputs

# Split the dataset into train and test
ds_split = ds['train'].train_test_split(train_size=1000, test_size=200)


train_dataset = ds_split['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=ds['train'].column_names, # Remove original columns if desired
)

eval_dataset = ds_split['test'].map(
    preprocess_function,
    batched=True,
    remove_columns=ds['train'].column_names, # Remove original columns if desired
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,1.610400
40,1.408100
60,1.397500
80,1.350100
100,1.275600
120,1.283100
140,1.263800
160,1.284300
180,1.378500
200,1.265500


TrainOutput(global_step=1000, training_loss=1.2235196266174317, metrics={'train_runtime': 2670.2628, 'train_samples_per_second': 0.374, 'train_steps_per_second': 0.374, 'total_flos': 1.867973611562496e+16, 'train_loss': 1.2235196266174317, 'epoch': 1.0})

In [ ]:
from peft import PeftModel

# Save LoRA adapters only
trainer.model.save_pretrained("gemma-2b-codeforces-cot-adapter")
tokenizer.save_pretrained("gemma-2b-codeforces-cot-adapter")


('gemma-2b-codeforces-cot-adapter/tokenizer_config.json',
 'gemma-2b-codeforces-cot-adapter/special_tokens_map.json',
 'gemma-2b-codeforces-cot-adapter/chat_template.jinja',
 'gemma-2b-codeforces-cot-adapter/tokenizer.model',
 'gemma-2b-codeforces-cot-adapter/added_tokens.json',
 'gemma-2b-codeforces-cot-adapter/tokenizer.json')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gemma-2b-cot-leetcode.zip to gemma-2b-cot-leetcode.zip


In [ ]:
!unzip gemma-2b-cot-leetcode.zip

Archive:  gemma-2b-cot-leetcode.zip
   creating: gemma-2b-cot-leetcode/
  inflating: gemma-2b-cot-leetcode/adapter_config.json  
  inflating: gemma-2b-cot-leetcode/adapter_model.safetensors  
  inflating: gemma-2b-cot-leetcode/chat_template.jinja  
  inflating: gemma-2b-cot-leetcode/README.md  
  inflating: gemma-2b-cot-leetcode/special_tokens_map.json  
  inflating: gemma-2b-cot-leetcode/tokenizer.json  
  inflating: gemma-2b-cot-leetcode/tokenizer.model  
  inflating: gemma-2b-cot-leetcode/tokenizer_config.json  


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "google/gemma-2-2b-it"   # Changed to match the likely base model of the adapter
lora_path = "gemma-2b-cot-leetcode"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype="auto",
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [ ]:
import torch

def generate(model, tokenizer, prompt,
             max_new_tokens=512,
             temperature=0.2,
             top_p=0.95,
             do_sample=False,
             num_return_sequences=1):
    """
    Robust generation wrapper:
    - moves input tensors to the model device,
    - uses no_grad,
    - sets pad_token_id/eos_token_id when needed,
    - decodes result cleanly.
    """
    # Ensure model on eval
    model.eval()

    # Build chat messages as you used elsewhere
    messages = [{"role": "user", "content": prompt}]
    tokenized = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    )

    # tokenized is a dict like {'input_ids': tensor(...), 'attention_mask': tensor(...)}
    # Move only tensors to the model device
    model_device = next(model.parameters()).device
    for k, v in tokenized.items():
        if isinstance(v, torch.Tensor):
            tokenized[k] = v.to(model_device)

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized.get("attention_mask", None)

    # safety: ensure pad_token_id/eos_token_id for generation
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token or tokenizer.eos_token_id
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    eos_id = tokenizer.eos_token_id

    gen_kwargs = dict(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences,
        pad_token_id=pad_id,
        eos_token_id=eos_id,
    )

    with torch.no_grad():
        outputs = model.generate(**gen_kwargs)

    # outputs shape: (num_return_sequences, seq_len)
    results = []
    # start index for generated tokens
    start_idx = input_ids.shape[-1]
    for i in range(outputs.shape[0]):
        gen_ids = outputs[i][start_idx:]
        text = tokenizer.decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        results.append(text)

    # if only one sequence requested, return the string
    if num_return_sequences == 1:
        return results[0]
    return results


In [ ]:
# ----------------------------
# Inference on a random example
# ----------------------------
import random

idx = random.randint(1001, len(ds['train'])-1)
test = ds['train'][idx]

print("\n⭐ TEST EXAMPLE:")
print(test["prompt"])

result = generate(model, tokenizer, prompt=test['prompt'], max_new_tokens=5000, temperature=0.1, do_sample=False)

print("\n⭐ MODEL OUTPUT:")
print(result)

print("\n⭐ EXPECTED OUTPUT:")
print(test["generation"])



⭐ TEST EXAMPLE:
You will be given a competitive programming problem.
Analyze the maximum input constraints and identify the optimal algorithmic approach and data structures needed to process the largest possible test cases within the time and memory limits, then explain why your chosen implementation strategy is the most efficient solution. Please reason step by step about your solution approach, then provide a complete implementation in C++17 that is thoroughly optimized for both speed and memory usage.

Your solution must read input from standard input (cin), write output to standard output (cout).
Do not include any debug prints or additional output.

Put your final solution within a single code block:
```cpp
<your code here>
```

# Problem

The on-board computer on Polycarp's car measured that the car speed at the beginning of some section of the path equals v1 meters per second, and in the end it is v2 meters per second. We know that this section of the route took exactly t secon